# Analyse exploratoire

Dans ce notebook, nous allons explorer la base de données disponible avec ce lien : https://www.kaggle.com/datasets/kazanova/sentiment140. Elle contient les tweets d'utilisateurs ainsi que le sentiment qu'il laisse transparaître, c'est à dire positif ou négatif

## Premier coup d'oeil

Commençons par survoler notre base de données. Le csv n'a pas de titre, nous allons donc les définir nous mêmes.

In [ ]:
import polars as pl

df = pl.read_csv("data/training.csv", encoding="latin-1", has_header=False)
df.columns = ["target", "id", "date", "query", "name", "content"]

df

On remarque que les colonnes ne sont pas nommées dans le fichier csv, ce qui a donc eu pour effet d'avoir utilisé les données de la première ligne comme titre. Vu qu'il y a plus d'un million de lignes, nous pourrions nous contenter de renommer chaque colonne sans récupérer la première ligne, mais même si ça aura peu d'impact sur notre programme, faisons les choses bien et récupérons là.

Maintenant que nous nous sommes occupés des titres, observons le nombre de valeurs différentes de chaque colonne.

In [ ]:
df.select(pl.all().n_unique())

La colonne "query" ne contient qu'une seule valeur: "NO_QUERY". Elle ne pourra donc pas nous aider à entraîner un modèle de machine learning, nous pouvons donc la supprimer.

In [ ]:
df = df.drop("query")

Un autre détail nous interpelle, il y a 1 600 000 lignes, mais pourtant il n'y a que 1 598 315 "id" différents. Assurons-nous donc qu'il n'y a pas de doublons.

In [ ]:
df.unique()

Les lignes sont pourtant toutes différentes. Regardons plus en détail deux lignes contenant le même id.

In [ ]:
dup_ids = df.filter(pl.col("id").is_duplicated()).sort("id")
dup_ids

On comprend mieux ce qu'il se passe. En fait les "id" en doublons sont les mêmes lignes mais avec deux target différentes, ce qui est une anomalie. Nous allons donc les supprimer, étant donné qu'elles ne concernent que 0,2% de nos données, ce n'est pas très grave. Profitons-en au passage pour changer ses valeurs de 0 et 4 au 0 et 1 traditonnel.

In [ ]:
dup_ids_list = dup_ids["id"].unique().implode()
df = df.filter(~df["id"].is_in(dup_ids_list))

df = df.with_columns(pl.col("target").replace(4, 1).alias("target"))
df

Maintenant que nous avons réglé ce problème d'ids récurrents, nous allons modifier le type de données de la colonne "date" de string à datetime, ce qui la rendra plus facile à manipuler.

In [ ]:
df = df.with_columns(pl.col("date").str.to_datetime("%a %b %d %H:%M:%S %Z %Y"))
df.head()

In [ ]:
df.describe()

On obtient quelques informations intéressantes. En premier lieu, notre "target" est bien distribué, ce qui évitera des biais lors des entraînements de nos modèles. Ensuite, le premier et le dernier tweet sont espacés de à peu près 11 semaines, mais que plus de 50% d'entre eux ont été récupérés sur une période de seulement 3 semaines. Il faudra être attentif à ça, car des tweets pourraient être biasiés à cause d'un événement. Et pour finir, on se rend compte en regardant "content" que certains messages sont très bizarres, potentiellement dus au formatage.

## Distribution

Commençons par visualiser la "target".

In [ ]:
import plotly.express as px

target_count = df["target"].value_counts()

fig = px.bar(
    target_count,
    x="target",
    y="count",
    title="Distribution de la target",
    width=800
)

grap_title = {
        "size":30,
        "weight": 600
	}

fig.update_layout(
	title_font = grap_title,
    title_x = 0.5,
    xaxis_tick0 = 0,
    xaxis_dtick = 1,
    yaxis_showticklabels = False
)

fig.show()

On remarque qu'elle est parfaitement distribué.

Ceci fait, penchons nous sur la distribution des dates.

In [ ]:
df_bins = (
    df.with_columns(
        pl.col("date").dt.truncate("1w").alias("week")
    )
    .group_by("week")
    .len()
)

fig = px.bar(
    df_bins,
    x="week",
    y="len",
    title="Distribution des dates"
)

fig.update_layout(
	title_font = grap_title,
    title_x = 0.5,
    xaxis_title="Date",
    yaxis_title="Fréquence"
)

fig.show()

On se rend compte que près de trois quarts des données ont été récupérées sur une période de seulement 4 semaines. Comme je l'ai dit précédemment, il faudra y être attentif.

## Contenu

Occupons-nous maintenant des lignes dont les tweets contiennent des caractères non ASCII. Effectivement, ils poseront problème au bon fonctionnement de nos modèles.

In [ ]:
df_ascii = df.filter(~df["content"].str.contains(r"^[\x00-\x7F]*$"))
pl.Config.set_fmt_str_lengths(50)
df_ascii["content"]

Il y a 14 500 lignes contenant des caractères non ASCII. C'est relativement peu, nous pouvons donc nous permettre de les supprimer. Cependant, en regardant ces lignes, on peut se rendre compte que tous les tweets ne sont pas en anglais. Cela risque d'être un problème et nous devons les trouver.

Tout d'abord, supprimons les lignes que nous venons de trouver.

In [ ]:
df = df.filter(~df["id"].is_in(df_ascii["id"].implode()))

Maintenant, réglons ce problème de langues. Pour ça nous allons installer la librairie "Pycld2". Elle permet d'automatiquement détecter les langues utilisées dans un texte.

In [ ]:
import pycld2 as cld2

def detect_lang_batch(batch):
    
    out = []
    for content in batch:
        
        try:
            _, _, details = cld2.detect(content)
            out.append(details[0][1])

        except Exception as e:
            out.append(f"__ERROR__:{type(e).__name__}:{str(e)[:200]}")

    return pl.Series(out)

df = df.with_columns(
    pl.col("content").map_batches(detect_lang_batch, return_dtype=pl.Utf8, returns_scalar=False).alias("lang_safe")
)

df.filter(pl.col("lang_safe").str.starts_with("__ERROR__"))

On se rend compte que quelques lignes comportent des erreurs d'encodage, mais il n'y en a que 16. Nous pouvons donc nous contenter de les supprimer.

Regardons maintenant le nombre de lignes ayant été détectées écrites en anglais.

In [ ]:
df["lang_safe"].value_counts().sort("count", descending=True)


Il restera plus de 1 500 000 lignes après la suppression des lignes non anglaises. C'est largement suffisant.

In [ ]:
df = df.filter(df["lang_safe"] == "en").drop("lang_safe")

Jetons un oeil à la longueur des tweets.

In [ ]:
content_length = df["content"].str.len_bytes()

fig = px.histogram(content_length, title="Distribution de la longueur des tweets")

fig.update_layout(
	title_font = grap_title,
    title_x = 0.5,
    xaxis_title="Date",
    yaxis_title="Fréquence"
)

fig.show()

On remarque que les tweets ont une longueur assez bien répartie autour de 40, avec un pic autour d'une longueur de 130 caractères.

Nous allons maintenant afficher un nuage de mots afin de visualiser les mots les plus utilisés. Avant ceci, nous allons bien faire attention à ne pas considérer les caractères, tels que ", qui ne sont pas très représentatifs du contenu des tweets.

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import html

wrd_cld_df = df.with_columns(pl.col("content").map_elements(html.unescape))

wrd_cld = WordCloud(width=800, height=400, background_color="white")
wrd_cld.generate(" ".join(wrd_cld_df["content"].to_list()))

plt.figure(figsize=(10, 10))

plt.imshow(wrd_cld, interpolation='bilinear')
plt.axis("off")
plt.title("Nuage de mots des tweets", fontsize=30, weight="bold", pad=20)
plt.show()

On remarque clairement des mots qui apparaissent très souvent mais ce qui serait encore mieux, étant donné que l'on veut lier un sentiment positif ou négatif à ces mots, serait qu'on divise notre tableau de données en deux avec les tweets positifs d'un côté et les tweets négatifs de l'autre et qu'on étudie la différence des mots retrouvés.

In [ ]:
wrd_cld_df_0 = wrd_cld_df.filter(pl.col("target") == 0)
wrd_cld_df_1 = wrd_cld_df.filter(pl.col("target") == 1)

wrd_cld_0 = WordCloud(width=800, height=400, background_color="white")
wrd_cld_0.generate(" ".join(wrd_cld_df_0["content"].to_list()))

wrd_cld_1 = WordCloud(width=800, height=400, background_color="white")
wrd_cld_1.generate(" ".join(wrd_cld_df_1["content"].to_list()))

plt.figure(figsize=(20, 10))

plt.subplot(1, 2, 1)
plt.imshow(wrd_cld_0, interpolation="bilinear")
plt.axis("off")
plt.title("Tweets négatifs", fontsize=30, weight="bold", pad=20)

plt.subplot(1, 2, 2)
plt.imshow(wrd_cld_1, interpolation="bilinear")
plt.axis("off")
plt.title("Tweets positifs", fontsize=30, weight="bold", pad=20)

plt.show()

Nous remarquons que certains mots se retrouvent à part égale dans les deux types de tweets (going, now, today...), mais aussi que certains mots sont clairement négatifs (work, want, miss...), et d'autres clairement positifs (love, thank, lol...). Ceci nous rassure sur la faisabilité de la mission.